In [93]:
import os 
import geopandas as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [94]:
data_dir = '/Users/aakash/Desktop/CIMIS/dailyStns/'
zipfiles = [os.path.join(data_dir,x) for x in os.listdir(data_dir) if x.endswith(".zip")]

In [95]:
outdirs = []
for i in zipfiles:
    print(os.path.split(i)[1][:-4])
    outdirs.append(os.path.join(data_dir,os.path.split(i)[1][:-4]))
    os.system('''unzip {} -d{}'''.format(i, os.path.join(data_dir,os.path.split(i)[1][:-4])))

dailyStns2008
dailyStns2009
dailyStns2018
dailyStns2002
dailyStns2016
dailyStns2017
dailyStns2003
dailyStns2015
dailyStns2001
dailyStns2000
dailyStns2014
dailyStns2010
dailyStns2004
dailyStns2005
dailyStns2011
dailyStns2007
dailyStns2013
dailyStns2012
dailyStns2006


In [96]:
outdirs.sort()
workingdir = outdirs[0]
dirfiles = [os.path.join(workingdir,x) for x in os.listdir(workingdir)]
df = pd.read_csv(dirfiles[55], header=None)

# # These cols are from the Readme
# columns = ['id', 'date', 'julian_date', 'ref_eto','ref_eto_qc', 'precip', 'precpi_qc', 'srad', 'srad_qc', 'vp', 'vp_qc', 'max_airtemp', 'max_airtemp_qc', 'min_airtemp', 
#  'min_airtemp_qc', 'avg_airtemp', 'avg_airtemp_qc', 'max_rh', 'max_rh_qc', 'min_rh', 'min_rh_qc', 'avg_rh', 'avg_rh_qc', 'dew_pt', 'dew_pt_qc', 'avg_windspeed',
#           'avg_windspeed_qc', 'wind_run', 'wind_run_qc', 'avg_soil_temp', 'avg_soil']

columns = ['id', 'date', 'julian_date', 'ref_eto_qc', 'ref_eto', 'precpi_qc','precip', 'srad_qc', 'srad',  'vp_qc', 'vp', 'max_airtemp_qc',  'max_airtemp', 'min_airtemp', 
 'min_airtemp_qc', 'avg_airtemp_qc','avg_airtemp',  'max_rh_qc','max_rh',  'min_rh_qc', 'min_rh',  'avg_rh_qc', 'avg_rh','dew_pt_qc',    'dew_pt',
           'avg_windspeed', 'avg_windspeed_qc', 'wind_run_qc', 'wind_run',   'avg_soil', 'avg_soil_temp']

df.columns = columns


In [97]:
dfs = []

for i in dirfiles:
    d = pd.read_csv(i, header = None)
    d.columns = columns
    dfs.append(d)

In [167]:
mdf = pd.concat(dfs)
mdf.date = pd.to_datetime(mdf.date)
mdf.set_index('date', inplace = True)
mdft = mdf.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


In [168]:
# Convert to float, add nans for missing data
et = []
for i in mdft.ref_eto:
    try:
        et.append(float(i))
    except:
        et.append(np.nan)

# add column to df
mdft['ref_eto_f'] = et
fin = mdft[mdft.ref_eto_qc == '*'] # Filter flagged observaitons
fdf = fin[fin['ref_eto_f'].notna()] # remove nans 

In [196]:
finals = {}
for i in fdf.id.unique():
    sdf = fdf[fdf.id == i]
    finals[i] = sdf.ref_eto_f.resample("M").mean()

In [197]:
finals

{19: date
 2000-01-31     66.290323
 2000-02-29    107.000000
 2000-03-31    192.833333
 2000-04-30    264.464286
 2000-05-31    277.838710
 2000-06-30    247.066667
 2000-07-31    218.548387
 2000-08-31    215.387097
 2000-09-30    213.733333
 2000-10-31    131.225806
 2000-11-30    128.000000
 2000-12-31     99.193548
 Freq: M, Name: ref_eto_f, dtype: float64, 150: date
 2000-01-31    132.129032
 2000-02-29    145.916667
 2000-03-31    166.250000
 2000-04-30    247.655172
 2000-05-31    282.222222
 2000-06-30    280.466667
 2000-07-31    276.419355
 2000-08-31    236.000000
 2000-09-30    200.172414
 2000-10-31    156.833333
 2000-11-30    150.583333
 2000-12-31    147.916667
 Freq: M, Name: ref_eto_f, dtype: float64, 30: date
 2000-01-31     71.838710
 2000-02-29     97.357143
 2000-03-31    182.000000
 2000-04-30    248.727273
 2000-05-31    287.300000
 2000-06-30    345.533333
 2000-07-31    331.866667
 2000-08-31    280.580645
 2000-09-30    233.896552
 2000-10-31    151.241379
 